# EDA

In [17]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

#importing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
logreg=LogisticRegression(solver="lbfgs")
from sklearn.model_selection import train_test_split
from sklearn import  metrics
from sklearn.metrics import accuracy_score

In [18]:
df = pd.read_csv("data/personal_transactions.csv")
df.Date = pd.to_datetime(df.Date)

In [19]:
df.head()

,Date,Description,Amount,Transaction Type,Category,Account Name
0,2018-01-01,Amazon,11.11,debit,Shopping,Platinum Card
1,2018-01-02,Mortgage Payment,1247.44,debit,Mortgage & Rent,Checking
2,2018-01-02,Thai Restaurant,24.22,debit,Restaurants,Silver Card
3,2018-01-03,Credit Card Payment,2298.09,credit,Credit Card Payment,Platinum Card
4,2018-01-04,Netflix,11.76,debit,Movies & DVDs,Platinum Card


In [20]:
df.shape

(806, 6)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 806 entries, 0 to 805
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              806 non-null    datetime64[ns]
 1   Description       806 non-null    object        
 2   Amount            806 non-null    float64       
 3   Transaction Type  806 non-null    object        
 4   Category          806 non-null    object        
 5   Account Name      806 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 37.9+ KB


In [22]:
df.describe()

,Amount
count,806.000000
mean,273.391489
std,667.630374
min,1.750000
25%,15.687500
50%,37.480000
75%,117.680000
max,9200.000000


In [23]:
debits = df[df["Transaction Type"] == 'debit']

In [24]:
def count_sum(data ,column: str, plot = "Pie"):
    by_column = data\
                .groupby(column)\
                .agg({"Transaction Type": "count", "Amount": "sum"})\
                .rename(columns={"Transaction Type": "Total"})\
                .reset_index()
    by_column.columns = [column, "Total", "Sum"]
    
    labels = by_column[column]

    fig = None
    
    if plot == 'Pie':
        fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Sum"],
                name="Sum"
            ),
        1, 1)
        fig.add_trace(
            go.Pie(
                labels=labels,
                values=by_column["Total"],
                name="Total"
            ),
        1, 2)
    elif plot == 'Scatter':
        fig = make_subplots(1, 2,
                    subplot_titles=['Sum', 'Total'])
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Sum"],
                name="Sum"
            ),
        1, 1)
        fig.add_trace(
            go.Scatter(
                x=labels,
                y=by_column["Total"],
                name="Total"
            ),
        1, 2)
        

    fig.update_layout(title_text=f"{column} Analysis")
    fig.show()
    
    return by_column

In [25]:
gastos_by_cat = count_sum(debits, "Category")
gastos_by_cat

,Category,Total,Sum
0,Alcohol & Bars,25,539.13
1,Auto Insurance,18,1350.00
2,Coffee Shops,31,115.54
3,Credit Card Payment,71,33041.36
4,Electronics & Software,4,719.00
5,Entertainment,1,9.62
6,Fast Food,16,330.63
7,Food & Dining,2,77.75
8,Gas & Fuel,52,1715.17
9,Groceries,105,2795.21


In [26]:
#null
df.isnull().sum()

Date                0
Description         0
Amount              0
Transaction Type    0
Category            0
Account Name        0
dtype: int64

In [27]:
#find the percentage of all missing values
features_with_null_values=[features for features in df.columns if df[features].isnull().sum()>1]

for feature in features_with_null_values:
    print(feature," : ",np.round(df[feature].isnull().mean(),4),"%")
          
print("\nfeature count with null values ",len(features_with_null_values))


feature count with null values  0


### Payment Category Destribution

In [28]:
df_category=df['Category'].value_counts().to_frame().reset_index().rename(columns={'index':'Category','Category':'count'})

fig = go.Figure(go.Bar(
    x=df_category['Category'],y=df_category['count'],
    marker={'color': df_category['count'], 
    'colorscale': 'Viridis'},  
    text=df_category['count'],
    textposition = "outside",
))
fig.update_layout(title_text='Category Distribution',xaxis_title="Category",yaxis_title="Count",title_x=0.5)
fig.show()

In [ ]:
X=df.drop(["ID","app_status"],axis="columns")